In [1]:
from collections import defaultdict
from itertools import cycle, islice

empty_row = lambda: ['|'] + 7*['.'] + ['|']

def make_rows():
    # storing all the air '.' and walls '|' is not very
    # efficient, but who cares?
    return defaultdict(empty_row)    

In [2]:
horiz_line = ((0, 0), (1, 0), (2, 0), (3, 0))
plus_shape = ((1, 0), (0, -1), (1, -1), (2, -1), (1, -2))
mirror_ell = ((2, 0), (2, -1), (2, -2), (1, -2), (0, -2))
vertl_line = ((0, 0), (0, -1), (0, -2), (0, -3))
block_shpe = ((0, 0), (1, 0), (0, -1), (1, -1))

shapes_nc = [horiz_line, plus_shape, mirror_ell, vertl_line, block_shpe] # no cycle
shapes = cycle(shapes_nc)

In [3]:
min(y for (x, y) in mirror_ell)
max(x for (x, y) in mirror_ell)

2

In [4]:
for s in islice(shapes, 5):
    cave = make_rows()
    for x, y in s:
        cave[y][x+1] = "#"
    rows = cave.keys()
    y0 = min(rows)
    y1 = max(rows)
    for y in range(y1, y0-1, -1):
        print("".join(cave[y]))
    print()

|####...|

|.#.....|
|###....|
|.#.....|

|..#....|
|..#....|
|###....|

|#......|
|#......|
|#......|
|#......|

|##.....|
|##.....|



In [5]:
jet_pattern = cycle(">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>")

cave = make_rows()
# make bottom
cave[0][0] = "+"
cave[0][8] = "+"
for x in range(1, 8):
    cave[0][x] = "-"

In [7]:
def print_cave(cave):
    rows = cave.keys()
    y0 = min(rows)
    y1 = max(rows)
    w = len(str(y1))
    form = "%" + "%d.d" % w

    for y in range(y1, y0-1, -1):
        print(form % y, "".join(cave[y]))

In [8]:
print_cave(cave)

0 +-------+


In [9]:
def run_single_rock(cave, rock, pattern):
    y = max(cave.keys()) + 4 - min(y for (x, y) in rock)
    x = 3
    width = max(x for (x, y) in rock)
    falling = True
    while falling:
        blow = next(pattern)
        if blow == "<":
            x1 = x - 1
        else:
            x1 = x + 1

        # test for horizontal movement    
        vals = [cave.get(y+dy, empty_row())[x1+dx] for (dx, dy) in rock]
        if all(v == '.' for v in vals):
            x = x1            
        
        y1 = y - 1
        # use get, so that we do not make new rows without purpose
        vals = [cave.get(y1+dy, empty_row())[x+dx] for (dx, dy) in rock]
        if all(v == '.' for v in vals):
            y = y1
        else:
            # rock cannot fall anymore to y1
            # -> insert it at old y position!
            falling = False
            for dx, dy in rock:
                cave[y+dy][x+dx] = '#'

In [10]:
for i in range(2022):
    run_single_rock(cave, next(shapes), jet_pattern)

max(cave.keys())

3068

In [11]:
# putting it all in one fucntion
def part1(pattern, num_rocks=2022):
    # we must declare shapes and jet_pattern again to restart the iterator
    shapes = cycle([horiz_line, plus_shape, mirror_ell, vertl_line, block_shpe])
    jet_pattern = cycle(pattern)

    cave = make_rows()
    # make bottom
    cave[0][0] = "+"
    cave[0][8] = "+"
    for x in range(1, 8):
        cave[0][x] = "-"

    for i in range(num_rocks):
        run_single_rock(cave, next(shapes), jet_pattern)

    return max(cave.keys()), cave


In [12]:
with open("input") as f:
    my_pattern = f.read().strip()

In [13]:
height, cave = part1(my_pattern)
height

3163

# Part 2

I think, we have to compress the cave once a complete row is 
covered with rocks.

```
|##.....|
|.###...|
|.....##|
|...####|
|#######|
```

First, I will try the easy route and delete only completely covered rows (with no vertical extension)


In [14]:
def run_single_rock2(cave, rock, pattern):
    y = max(cave.keys()) + 4 - min(y for (x, y) in rock)
    x = 3
    width = max(x for (x, y) in rock)
    falling = True
    while falling:
        n, blow = next(pattern)
        if blow == "<":
            x1 = x - 1
        else:
            x1 = x + 1

        # test for horizontal movement    
        vals = [cave.get(y+dy, empty_row())[x1+dx] for (dx, dy) in rock]
        if all(v == '.' for v in vals):
            x = x1            
        
        y1 = y - 1
        # use get, so that we do not make new rows without purpose
        vals = [cave.get(y1+dy, empty_row())[x+dx] for (dx, dy) in rock]
        if all(v == '.' for v in vals):
            y = y1
        else:
            # rock cannot fall anymore to y1
            # -> insert it at old y position!
            falling = False
            for dx, dy in rock:
                cave[y+dy][x+dx] = '#'

    # return position of placement
    return (x, y, n)


I have to find a good criterium to tell if a row is full.
We will only look at a couple of rows at the same time

If at the same pattern position n and the same shape
we have the same cave configuration (from the last complete row)
the configuration will be the same, as everything is deterministic.

In this case we can skip the simulation of falling rocks and
just see how many cycles we will run in the end.

In [15]:
def cave_hash(cave):
    # hash for cave
    y0, y1 = min(cave.keys()), max(cave.keys())
    return "".join("".join(cave[y]) for y in range(y0, y1+1))

# putting it all in one fucntion
def part2_a(pattern, num_rocks=2022, watch_cycles=True):
    # we must declare shapes and jet_pattern again to restart the iterator
    shapes = cycle([horiz_line, plus_shape, mirror_ell, vertl_line, block_shpe])
    jet_pattern = cycle(enumerate(pattern))

    cave = make_rows()
    # make bottom
    cave[0][0] = "+"
    cave[0][8] = "+"
    for x in range(1, 8):
        cave[0][x] = "-"

    seen = {}

    for i in range(num_rocks):
        shape = next(shapes)
        # n is the current position in the repeating blow pattern
        x, y, n = run_single_rock2(cave, shape, jet_pattern)

        # naive way to look for closed line
        # this might not find all closed lines. But it is enough for
        # cylce detection if we find at least some.
        if True:
            for y1 in sorted(set(y + dy for dx, dy in shape))[::-1]:
                row = cave.get(y1, empty_row())
                found_hole = False
                for xx, ch in enumerate(row):
                    if ch == ".":
                        # is this hole covered from above?
                        if cave.get(y1+1, empty_row())[xx] == '.':
                            found_hole = True
                            break

                if not found_hole:
                    # this is a close line
                    #print("closed", i, "at", y1)
                    rows_below = [y2 for y2 in cave.keys() if y2 < y1]
                    #for yy in range(y1+4, y1-5, -1):
                    #    print("*" if yy == y1 else " ", "".join(cave[yy]))

                    for y3 in rows_below:
                        del cave[y3]


        if watch_cycles:
            if n < 5:
                # at the beginning of a new wave of blows
                s = hash(cave_hash(cave))
                if (n, shape, s) in seen:
                    old_i, old_cave = seen[n, shape, s]
                    print("seen", i, "at", old_i, i - old_i)
                    print("cycle detected!")
                    cycle_height = max(cave.keys()) - max(old_cave.keys())
                    return max(cave.keys()), cave, seen, i - old_i, i, cycle_height
                else:
                    print("***NEW***", i, end=" ")
                    seen[n, shape, s] = [i, dict(cave)]
                    print("new config", n, shape, s)


    return max(cave.keys()), cave, seen, 0, i, 0

In [16]:
height, cave, seen, cycle_length, num_fallen, cycle_height = part2_a(my_pattern, num_rocks=3600)
#print_cave(cave)
height

***NEW*** 0 new config 3 ((0, 0), (1, 0), (2, 0), (3, 0)) -3885326096268831555
***NEW*** 1698 new config 4 ((0, 0), (0, -1), (0, -2), (0, -3)) 4372129826719589664
seen 3413 at 1698 1715
cycle detected!


5328

In [17]:
s1 = ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
len(s1)

40

In [18]:
height, cave, seen, cycle_length, num_fallen, cycle_height = part2_a(">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>", num_rocks=600)
print_cave(cave)
height

***NEW*** 0 new config 3 ((0, 0), (1, 0), (2, 0), (3, 0)) 6845641884179467986
***NEW*** 8 new config 2 ((0, 0), (0, -1), (0, -2), (0, -3)) 8587835204208663678
***NEW*** 14 new config 1 ((0, 0), (1, 0), (0, -1), (1, -1)) 1229170892121215119
***NEW*** 22 new config 0 ((2, 0), (2, -1), (2, -2), (1, -2), (0, -2)) 3718111562597264209
***NEW*** 29 new config 4 ((0, 0), (1, 0), (0, -1), (1, -1)) -250328460786536390
***NEW*** 36 new config 2 ((1, 0), (0, -1), (1, -1), (2, -1), (1, -2)) -3296577412329830024
***NEW*** 43 new config 1 ((0, 0), (0, -1), (0, -2), (0, -3)) -2871009156588369442
***NEW*** 49 new config 1 ((0, 0), (1, 0), (0, -1), (1, -1)) 1818938777701900558
***NEW*** 57 new config 0 ((2, 0), (2, -1), (2, -2), (1, -2), (0, -2)) -1964602151475633301
seen 64 at 29 35
cycle detected!
104 |#......|
103 |#......|
102 |#.#....|
101 |#.#....|
100 |####...|
 99 |..#####|


104

The `cycle_lenght` is the number of rocks, that will fall until a configuration repeats.

In [19]:
cycle_length, num_fallen, cycle_height

(35, 64, 53)

In [20]:
num = 1000000000000

In [21]:
num_left = num - num_fallen
full_cycles_left = num_left // cycle_length
single_rocks_left = num_left - full_cycles_left * cycle_length

full_cycles_left, single_rocks_left

(28571428569, 21)

Now let the last single rocks fall and put it all in one function:

In [22]:
def part2_b(cave, shapes, jet_pattern, num_rocks=2022, watch_cycles=True):
    seen = {}

    for i in range(num_rocks):
        shape = next(shapes)
        # n is the current position in the repeating blow pattern
        x, y, n = run_single_rock2(cave, shape, jet_pattern)

        # naive way to look for closed line
        # this might not find all closed lines. But it is enough for
        # cylce detection if we find at least some.
        if True:
            for y1 in sorted(set(y + dy for dx, dy in shape))[::-1]:
                row = cave.get(y1, empty_row())
                found_hole = False
                for xx, ch in enumerate(row):
                    if ch == ".":
                        # is this hole covered from above?
                        if cave.get(y1+1, empty_row())[xx] == '.':
                            found_hole = True
                            break

                if not found_hole:
                    # this is a close line
                    #print("closed", i, "at", y1)
                    rows_below = [y2 for y2 in cave.keys() if y2 < y1]
                    #for yy in range(y1+4, y1-5, -1):
                    #    print("*" if yy == y1 else " ", "".join(cave[yy]))

                    for y3 in rows_below:
                        del cave[y3]

        if watch_cycles:
            if n < 5:
                # at the beginning of a new wave of blows
                s = hash(cave_hash(cave))
                if (n, shape, s) in seen:
                    old_i, old_cave = seen[n, shape, s]
                    print("seen", i, "at", old_i, i - old_i)
                    print("cycle detected!")
                    cycle_height = max(cave.keys()) - max(old_cave.keys())
                    return max(cave.keys()), cave, seen, i - old_i, i, cycle_height
                else:
                    print("***NEW***", i, end=" ")
                    seen[n, shape, s] = [i, dict(cave)]
                    #print("new config", n, shape, s)

    return max(cave.keys()), cave, seen, 0, i, 0


def part2(pattern, num_rocks=2022):
    # we must declare shapes and jet_pattern again to restart the iterator
    shapes = cycle([horiz_line, plus_shape, mirror_ell, vertl_line, block_shpe])
    jet_pattern = cycle(enumerate(pattern))

    cave = make_rows()
    # make bottom
    cave[0][0] = "+"
    cave[0][8] = "+"
    for x in range(1, 8):
        cave[0][x] = "-"

    height, cave, seen, cycle_length, num_fallen, cycle_height = part2_b(cave, shapes, jet_pattern, num_rocks)
    
    num_left = num_rocks - num_fallen
    full_cycles_left = num_left // cycle_length
    single_rocks_left = num_left - full_cycles_left * cycle_length

    height, cave, _, _, _, _ = part2_b(cave, shapes, jet_pattern, single_rocks_left, watch_cycles=False)

    # dont know where the -1 comes from. Just got this from comparing the example ouput
    return height + full_cycles_left*cycle_height - 1   

In [23]:
part2(">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>", num_rocks=1000000000000)

***NEW*** 0 ***NEW*** 8 ***NEW*** 14 ***NEW*** 22 ***NEW*** 29 ***NEW*** 36 ***NEW*** 43 ***NEW*** 49 ***NEW*** 57 seen 64 at 29 35
cycle detected!


1514285714288

In [24]:
part2(my_pattern, num_rocks=1000000000000)

***NEW*** 0 ***NEW*** 1698 seen 3413 at 1698 1715
cycle detected!


1560932944615